In [1]:
from keras.models import Sequential
from keras.layers import Dense
from keras.models import model_from_json
import numpy as np
import os
import pickle
from sklearn.pipeline import make_pipeline
from lime import lime_text
from lime.lime_text import LimeTextExplainer
import warnings
from nltk.corpus import stopwords as sw
from string import punctuation
import json
import re

warnings.filterwarnings('ignore')

C:\Users\antoi\Anaconda3\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Set directory path
os.chdir('./Saved model')

# Text preprocessing functions 

In [3]:
def clean_text(text):
    characters_to_delete = ['«','»',"l’","n’","d’","s’",'qu’','≥',"l'","n'","d'","s'","qu'"]
    for c in characters_to_delete:
        text = text.replace(c,"")
       
    text = ''.join([i for i in text if not i.isdigit()])
    text = re.sub("\((.*?)\)","",text)
    text = ' '.join(text.split())
    text = str.lower(text)
    text = ''.join(c for c in text if c not in punctuation or c=="-")    
    return text

with open('../stopwords.json', 'r', encoding='utf-8') as f:
    stopwords = json.loads(f.read())
nltk_words = list(sw.words('english'))
stopwords.extend(nltk_words)

def remove_stopwords(word_list, stopwords):
    return [word for word in word_list if word not in stopwords]

# Load trained model and TF-IDF Vectorizers 

In [4]:
json_file = open('model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("model.h5")
print("Loaded model from disk")
 
# evaluate loaded model on test data
loaded_model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

with open('vectorizer.pk', 'rb') as f:
    tfidf = pickle.load(f)

Loaded model from disk
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [5]:
# creates the pipeline object for inference
class DenseTransformer():
    def transform(self, X, y=None, **fit_params):
        return X.todense()

    def fit_transform(self, X, y=None, **fit_params):
        self.fit(X, y, **fit_params)
        return self.transform(X)

    def fit(self, X, y=None, **fit_params):
        return self  
c = make_pipeline(tfidf, DenseTransformer(),loaded_model)


In [6]:
class_names = ['ACTU_MEDICALES', 'CARDIOLOGY', 'DERMATOLOGY', 'ENDOCTRINOLOGY', 'GASTROENTEROLOGY', 'GENERAL_MEDICINE', 'INFECTIOUSDISEASES', 'NEPHROLOGY', 'NEUROLOGY', 'OBSTETRICS', 'ONCOLOGY', 'PEDIATRICS', 'PHARMACISTS', 'PSYCHIATRY', 'PULMONOLOGY', 'RHEUMATOLOGY', 'UROLOGY']
explainer = LimeTextExplainer(class_names=class_names)

def predict_classes(text):
    exp = explainer.explain_instance(text, c.predict_proba, num_features=12, top_labels=3)
    exp.show_in_notebook()
    return np.array(class_names)[c.predict_proba([text]).flatten() > 0.45]

# Class predictions and related words

In [7]:
sample_text = open('../sample_text.txt', 'r').read()

# preprocess text
sample_text = " ".join(remove_stopwords(clean_text(sample_text).split(), stopwords))

predict_classes(sample_text)

InternalError: Blas GEMM launch failed : a.shape=(32, 3487), b.shape=(3487, 256), m=32, n=256, k=3487
	 [[Node: dense_1/MatMul = MatMul[T=DT_FLOAT, transpose_a=false, transpose_b=false, _device="/job:localhost/replica:0/task:0/device:GPU:0"](_arg_dense_1_input_0_0/_69, dense_1/kernel/read)]]
	 [[Node: activation_5/Sigmoid/_75 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_100_activation_5/Sigmoid", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:CPU:0"]()]]

Caused by op 'dense_1/MatMul', defined at:
  File "C:\Users\antoi\Anaconda3\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\antoi\Anaconda3\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\antoi\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_i